In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import skorch
import torchvision.datasets as dset
import torchvision.transforms as T
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

import numpy as np

In [3]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cpu


In [4]:
#load the data
x = np.load("./large_data/X.npy")
y = np.load("./large_data/y.npy")

x_full = torch.from_numpy(x)
y_full = torch.from_numpy(y)
x_full = x_full.to(device=device, dtype=dtype)
y_full = y_full.to(device=device, dtype=torch.long)

In [28]:
#split the data
x = x_full[30:70,:,:,:]
y = y_full[30:70]

print(x.shape)
print(y.shape)

x_Test = x_full[0:29,:,:,:]
y_Test = y_full[0:29]

print(x_Test.shape)
print(y_Test.shape)

torch.Size([40, 512, 512, 62])
torch.Size([40, 1])
torch.Size([29, 512, 512, 62])
torch.Size([29, 1])


In [6]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)


In [29]:
channel_1 = 64
channel_2 = 64
channel_3 = 128

learning_rate = 2.5e-3

in_channel = 62
num_classes = 3

# model = nn.Sequential(
#     nn.Conv2d(in_channel, channel_1, 5, padding=2),
#     nn.BatchNorm2d(channel_1),
#     nn.ReLU(),
#     nn.Conv2d(channel_1, channel_2, 3, padding=1),
#     nn.BatchNorm2d(channel_2),
#     nn.ReLU(),
#     nn.MaxPool2d(2),
#     Flatten(),
#     nn.Linear((32*512*512)/2, num_classes)
# )
# model = model.to(device=device)

resnet18 = models.resnet18(pretrained=True)
model = resnet18

model.conv1 = nn.Conv2d(in_channel, 64, kernel_size=7, stride=2, padding=3, bias=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs*100, num_classes)

from skorch import NeuralNetClassifier

net = NeuralNetClassifier(
    module=model,
    criterion = nn.CrossEntropyLoss,
    optimizer=optim.Adam,
    train_split=None,
    max_epochs=5,
    lr = learning_rate,
    warm_start = True,
    device = device
)

In [30]:
from sklearn.model_selection import KFold
from sklearn import metrics

# for epoch in range(5):
kf = KFold(n_splits=3, shuffle = True)
accuracies=[]
for train_index, test_index in kf.split(x):
    accuracies=[]
    (N, C1, C2, S) = x.shape # 17, 62, 512, 512
    x = x.reshape((N, 62, 512, 512))
    y = y.reshape((y.shape[0]))
    print(x.shape)
    print(y.shape)
    print(train_index)
    print(test_index)
    xk_train, xk_test = x[train_index], x[test_index]
    yk_train, yk_test = y[train_index], y[test_index]
    net.fit(xk_train,yk_train)
    y_pred = net.predict(xk_test)
    acc = metrics.accuracy_score(yk_test, y_pred)
    accuracies.append(acc)
    print('FinalAccuracy %.4f' % (np.mean(accuracies)))

torch.Size([40, 62, 512, 512])
torch.Size([40])
[ 2  4  8  9 10 11 12 14 15 16 18 19 21 22 23 25 26 27 28 30 31 33 36 37 38
 39]
[ 0  1  3  5  6  7 13 17 20 24 29 32 34 35]
  epoch    train_loss       dur
-------  ------------  --------
      1        0.9912  122.7344
      2       83.2823  115.4586
      3       17.9000  108.4039
      4        6.7772  111.2738
      5        4.4439  109.1874
FinalAccuracy 0.2143
torch.Size([40, 62, 512, 512])
torch.Size([40])
[ 0  1  3  5  6  7  8  9 10 13 15 17 18 19 20 22 24 26 29 30 31 32 33 34 35
 36 39]
[ 2  4 11 12 14 16 21 23 25 27 28 37 38]
      6        4.6350  110.7961
      7        3.4626  107.3966
      8        1.5131  105.9109
      9        2.1221  106.3230
     10        2.1959  112.1558
FinalAccuracy 0.4615
torch.Size([40, 62, 512, 512])
torch.Size([40])
[ 0  1  2  3  4  5  6  7 11 12 13 14 16 17 20 21 23 24 25 27 28 29 32 34 35
 37 38]
[ 8  9 10 15 18 19 22 26 30 31 33 36 39]
     11      114.9111  107.9736
     12       41.2930  

In [ ]:
(N, C1, C2, S) = x_Test.shape
x_Test = x_Test.reshape((N, S, C1, C2))
y_pred_test = net.predict(x_Test)
acc = metrics.accuracy_score(y_Test, y_pred_test)
print('TestAccuracy %.4f' % (acc))